# Process Data

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
text_file = open("spending.txt", "r")
lines = text_file.read().split('\n')
print(lines)
print(len(lines))
text_file.close()

FileNotFoundError: ignored

In [ ]:
months = {
    'jan':1,
    'feb':2,
    'mar':3,
    'apr':4,
    'mei':5,
    'may':5,
    'jun':6,
    'jul':7,
    'aug':8,
    'agu':8,
    'sep':9,
    'oct':10,
    'okt':10,
    'nov':11,
    'des':12,
    'dec':12
}
def convMonth(text):
  for x in months.keys():
    if x in text:
      return months[x]
  return 0

In [ ]:
def buat(x):
  global year,tanggal
  data = []
  x = x.lower().strip()
  if "----" in x or x == '' or x.isalnum() or "————" in x:
    #print(tanggal)
    return
  elif ':' not in x:
    print(x,tanggal)
    if (tanggal[1] == 12) and ('jan' in x):
      year= year+1
    tanggal = x.split(" ")
    tanggal[0] = int(tanggal[0])
    tanggal[1] = convMonth(tanggal[1])
    tanggal.append(year)
    #print(tanggal, "arr")
    return
  else:
    if '(' not in x:
      x = x + "(cash)"
    data = x.lower().replace("(",":").replace(")","").split(':')
    data = [el.strip() for el in data]
    data[0] = float(data[0][:-1])*1000
    return data + tanggal

In [ ]:
df = pd.DataFrame(columns=['price','name','method','day', 'month', 'year'])
year = 2019
tanggal = [0,0,0]
for x in lines:
  x = x.replace(',',".")
  row = buat(x)
  if row != None:
    new_df = pd.DataFrame([row],columns=['price','name','method','day', 'month', 'year'])
    df = pd.concat([df,new_df], ignore_index=True)
df

NameError: ignored

In [ ]:
df.to_csv("hasil.csv", index=False)

In [ ]:
len([1 for x in lines if ":" in x])

# Retrieve Data

In [ ]:
import pandas as pd
import numpy as np
import requests
import plotly.graph_objs as go
import plotly.express as px
import json
import pprint

In [ ]:
DATABASE_ID = "24f44f4bde444255847ebb05354fd7f9"
NOTION_URL = 'https://api.notion.com/v1/databases/'
class NotionSync:
    def __init__(self):
        pass    

    def query_databases(self,integration_token="secret_INIpbX6eKWFhV8lkWC5QBdmnFJYWJgK7vYOF24cmAG5",notion_version="2022-02-22"):
        database_url = NOTION_URL + DATABASE_ID + "/query"
        response = requests.post(database_url, headers={"Authorization": f"{integration_token}","Notion-Version":f"{notion_version}"})
        if response.status_code != 200:
            raise Exception(f'Response Status: {response.status_code} {response.text}')
        else:
            data = [response.json()]
            while response.json()['has_more']:
              cur = response.json()['next_cursor']
              response = requests.post(database_url, json={'start_cursor':cur}, headers={"Authorization": f"{integration_token}","Notion-Version":f"{notion_version}"})
              # print(response.json())
              data.append(response.json())
            return data
    
    def get_projects_titles(self,data_json):
        return list(data_json["results"][0]["properties"].keys())
    

    def get_projects_data(self,data_json,projects):
        data = []
        for i in data_json:
          res = i["results"]
          for val in res:
            row = []
            el = val['properties']
            for p in projects:
              tipe = el[p]['type']
              # print(tipe)
              # pprint.PrettyPrinter(indent=4).pprint(el[p][tipe])
              if tipe == 'formula':
                key = el[p][tipe]['type']
                if key == 'date':
                  row.append(el[p][tipe][key]['start'])
                else:
                  row.append(el[p][tipe][key])
              elif tipe =='title':
                row.append(el[p][tipe][0]['plain_text'])
              elif tipe =='select':
                # print(el[p][tipe])
                row.append(el[p][tipe]['name'])
              elif tipe == 'rich_text':
                # print(el[p][tipe])
                row.append(el[p][tipe][0]['text']['content'])
              else:
                row.append(el[p][tipe])
            data.append(row)
        return data

nsync = NotionSync()
data = nsync.query_databases()
projects = nsync.get_projects_titles(data[0])
data = nsync.get_projects_data(data,projects)
money_df = pd.DataFrame(data, columns = projects)
money_df.head()

,month,harga,Category,price,Property,method,month year,year,day,Date,Name
0,0,35.5k,food and beverages,35500,2022-03-25T12:29:00.000Z,cash,March 2022,0,0,2022-03-25T12:29:00.000+00:00,Food
1,0,23k,food and beverages,23000,2022-03-23T10:51:00.000Z,cash,March 2022,0,0,2022-03-23T10:51:00.000+00:00,Food and snacks
2,0,6k,food and beverages,6000,2022-03-21T15:04:00.000Z,cash,March 2022,0,0,2022-03-21T15:04:00.000+00:00,Food
3,0,42k,food and beverages,42000,2022-03-20T16:01:00.000Z,cash,March 2022,0,0,2022-03-20T16:01:00.000+00:00,Food
4,0,3k,lifestyle,3000,2022-03-19T02:43:00.000Z,cash,March 2022,0,0,2022-03-19T02:43:00.000+00:00,Conditioner


In [ ]:
money_df.shape

In [ ]:
money_df.to_excel("moneybook.xlsx")

# Categorize Data

In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.3 MB 10.5 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=49f3a9c081a1d3f59bb7c52df2b36f56c54a12a242f367f3d5dffccb052a1421
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
from googletrans import Translator 

#simple function to detect and translate text 
def detect_and_translate(text,target_lang):
    tl = Translator()
    result_lang = tl.detect(text['Name'])
    print(text['Name'], result_lang)
    result_lang = result_lang.lang if type(result_lang.lang) == str else result_lang.lang[0]
    if result_lang not in ['id','ja','ms','jw']:
        print(result_lang,target_lang)
        text['lang'] = target_lang if result_lang==target_lang else np.NaN
        text['translation'] = text['Name']
        #print(text)
        return text 
    
    else:
        text['translation'] = tl.translate(text['Name'],src=result_lang,dest=target_lang).text
        text['lang'] = target_lang
        return text

In [ ]:
text_df = pd.DataFrame(money_df['Name'].apply(lambda x: x.lower()), columns=['Name','translation','lang'])
text_df = text_df.drop_duplicates(subset=['Name'], ignore_index=True)

In [ ]:
text_df.describe()

,Name,translation,lang
count,179,0,0
unique,179,0,0
top,kipas angin,NaN,NaN
freq,1,NaN,NaN


In [ ]:
Translator().detect('Buku').lang

'id'

In [ ]:
text_df = text_df.apply(detect_and_translate, axis=1, args=('en',))

kipas angin Detected(lang=['id', 'ms'], confidence=[0.5857864, 0.41421357])
topup gopay Detected(lang=cs, confidence=0.4533093)
cs en
food Detected(lang=en, confidence=1)
en en
sisir butterfly Detected(lang=id, confidence=0.76794827)
lapangan Detected(lang=id, confidence=0.68774706)
drinks Detected(lang=en, confidence=1)
en en
eggs Detected(lang=en, confidence=1)
en en
milk Detected(lang=en, confidence=0.9869792)
en en
bedak muka Detected(lang=id, confidence=0.805886)
steam wallet Detected(lang=en, confidence=1)
en en
topup dana Detected(lang=id, confidence=0.88673145)
snacks Detected(lang=en, confidence=0.9356111)
en en
laundry Detected(lang=en, confidence=0.98307294)
en en
hp realme gt master Detected(lang=en, confidence=0.91365755)
en en
water Detected(lang=en, confidence=0.98307294)
en en
buku Detected(lang=id, confidence=0.7782101)
masjid Detected(lang=ms, confidence=0.30078125)
biaya admin Detected(lang=id, confidence=0.97744507)
food n drink Detected(lang=en, confidence=1)
en en

In [ ]:
text_df.loc[text_df['lang']!='en'].count()

Name           21
translation    21
lang            0
dtype: int64

In [ ]:
text_df.loc[text_df['lang']!='en']

,Name,translation,lang
1,topup gopay,topup gopay,NaN
25,es,es,NaN
34,bultang,bultang,NaN
42,ovo,ovo,NaN
59,welkin gi,welkin gi,NaN
62,topup ovo,topup ovo,NaN
63,tws,tws,NaN
64,tas,tas,NaN
73,koi,koi,NaN
83,shopee,shopee,NaN


In [ ]:
text_df.to_csv('processed.csv')

In [ ]:
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import re
import string
import nltk
import numpy as np
import pandas as pd

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
import gensim.downloader as api
glove = api.load('glove-wiki-gigaword-300')

from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

nltk.download("stopwords")
nltk.download('punkt')

[==================================================] 100.0% 376.1/376.1MB downloaded
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_csv('done_processed.csv')
text_df = df['translation'].drop_duplicates().to_frame(name='text')
text_df.head()

,text
0,laundry
1,food
2,drinks
3,snacks
4,field


In [ ]:
def clean_text(text, tokenizer, stopwords):
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [ ]:
text_df['tokens'] = text_df['text'].apply(lambda x: clean_text(x, word_tokenize, stopwords.words('english')))

In [ ]:
text_df

,text,tokens
0,laundry,[laundry]
1,food,[food]
2,drinks,[drinks]
3,snacks,[snacks]
4,field,[field]
...,...,...
170,father's phone and 1 year warranty,"[fathers, phone, year, warranty]"
171,citrus,[citrus]
172,stylus pen,"[stylus, pen]"
173,member card,"[member, card]"


In [ ]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model:
                try:
                    vectors.append(model[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(text_df['tokens'], model=glove)
len(vectorized_docs), len(vectorized_docs[0])

(163, 300)

In [ ]:
def mbkmeans_clusters(
	X, 
    k, 
    mb, 
    print_silhouette_values, 
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_docs,
    k=10,
    mb=100,
    print_silhouette_values=True,
)
text_df['cluster'] = cluster_labels

For n_clusters = 10
Silhouette coefficient: 0.04
Inertia:4121.769238978317
Silhouette values:
    Cluster 9: Size:2 | Avg:1.00 | Min:1.00 | Max: 1.00
    Cluster 0: Size:3 | Avg:0.39 | Min:0.15 | Max: 0.50
    Cluster 3: Size:5 | Avg:0.26 | Min:0.10 | Max: 0.38
    Cluster 4: Size:10 | Avg:0.13 | Min:0.05 | Max: 0.22
    Cluster 7: Size:5 | Avg:0.10 | Min:-0.06 | Max: 0.25
    Cluster 6: Size:20 | Avg:0.10 | Min:-0.02 | Max: 0.21
    Cluster 1: Size:4 | Avg:0.05 | Min:-0.01 | Max: 0.11
    Cluster 5: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 2: Size:104 | Avg:-0.02 | Min:-0.13 | Max: 0.07
    Cluster 8: Size:9 | Avg:-0.06 | Min:-0.25 | Max: 0.07


In [ ]:
text_df.groupby('cluster').sample(1)

,text,tokens,cluster
157,tools hardware,"[tools, hardware]",0
57,bag,[bag],1
22,if else punishment,"[else, punishment]",2
159,eggs,[eggs],3
96,transport museum ticket,"[transport, museum, ticket]",4
118,pants,[pants],5
13,drink,[drink],6
101,competition,[competition],7
140,roxy sandals,"[roxy, sandals]",8
4,field,[field],9


In [ ]:
category = ['Food and Beverages', 'Travel', 'Utilities','Healthcare','Savings & Debt', 
            'Entertainment','Lifestyle', 'Miscellaneous', 'Gadget']
category_tokens = [clean_text(x, word_tokenize, stopwords.words('english')) for x in category]

In [ ]:
category_tokens

[['food', 'beverages'],
 ['travel'],
 ['utilities'],
 ['healthcare'],
 ['savings', 'debt'],
 ['entertainment'],
 ['lifestyle'],
 ['miscellaneous'],
 ['gadget']]

In [ ]:
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
dictionary = Dictionary(text_df['tokens'].tolist()+category_tokens)
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

100%|██████████| 228/228 [00:14<00:00, 16.14it/s]


In [ ]:
[tfidf[dictionary.doc2bow(x)] for x in category_tokens]

[[(1, 0.5461163024151554), (220, 0.8377093674039932)],
 [(68, 1.0)],
 [(221, 1.0)],
 [(222, 1.0)],
 [(101, 0.5899725725570921), (223, 0.8074232865296659)],
 [(224, 1.0)],
 [(225, 1.0)],
 [(226, 1.0)],
 [(227, 1.0)]]

In [ ]:
pd.DataFrame(doc_similarity_scores, index=text_df['text'], columns=category)

NameError: ignored

In [ ]:
query_tf = [tfidf[dictionary.doc2bow(x)] for x in text_df['tokens'].tolist()]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in category_tokens]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
docs = text_df['text'].tolist()
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {docs[idx]}')

TypeError: ignored

# Visualize Data

In [ ]:
!pip install streamlit
!pip install pyngrok==4.1.1
!pip install millify

In [ ]:
from pyngrok import ngrok

In [ ]:
!ngrok authtoken auth_token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
money_df['Date'] = pd.to_datetime(money_df['Date'],format='%Y-%m-%dT%H:%M:%S.%f%z')

In [ ]:
curr = pd.Timestamp.now()
last_month = curr.replace(day=1) - pd.Timedelta(days=1)
money_df.loc[money_df['Date'].dt.to_period(freq='M')== curr.to_period(freq='M') ].reset_index()

Timestamp('2022-02-28 09:07:21.705745')

In [ ]:
from millify import millify
import plotly.express as px
import plotly.graph_objects as go

curr = pd.Timestamp.now()
last_month = curr.replace(day=1) - pd.Timedelta(days=1)
df = money_df.loc[money_df['Date'].dt.to_period(freq='M')== curr.to_period(freq='M') ].reset_index()
last_month_df =  money_df.loc[money_df['Date'].dt.to_period(freq='M')== last_month.to_period(freq='M') ].reset_index()

fig = px.line(df.groupby(pd.Grouper(key='Date',freq='W'))['price'].sum().reset_index(), 
								x="Date", y="price", 
								labels={'price':'Total Spendings'})
fig.add_scatter(x=last_month_df['Date'], y=last_month_df['price'], mode='lines', hoverinfo="text")
fig.show()

In [ ]:
%%writefile app.py
from millify import millify
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
import pandas as pd
import numpy as np
import requests
PAGE_CONFIG = {"page_title":"Spendings Report","page_icon":":dollar:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

def getdata():
	DATABASE_ID = database_id
	NOTION_URL = notion_url
	class NotionSync:
			def __init__(self):
					pass    
			def query_databases(self,integration_token=secret_token,notion_version=notion_vers):
					database_url = NOTION_URL + DATABASE_ID + "/query"
					response = requests.post(database_url, headers={"Authorization": f"{integration_token}","Notion-Version":f"{notion_version}"})
					if response.status_code != 200:
							raise Exception(f'Response Status: {response.status_code} {response.text}')
					else:
							data = [response.json()]
							while response.json()['has_more']:
								cur = response.json()['next_cursor']
								response = requests.post(database_url, json={'start_cursor':cur}, headers={"Authorization": f"{integration_token}","Notion-Version":f"{notion_version}"})
								# print(response.json())
								data.append(response.json())
							return data
			def get_projects_titles(self,data_json):
					return list(data_json["results"][0]["properties"].keys())
			def get_projects_data(self,data_json,projects):
					data = []
					for i in data_json:
						res = i["results"]
						for val in res:
							row = []
							el = val['properties']
							for p in projects:
								tipe = el[p]['type']
								# print(tipe)
								# pprint.PrettyPrinter(indent=4).pprint(el[p][tipe])
								if tipe == 'formula':
									key = el[p][tipe]['type']
									if key == 'date':
										row.append(el[p][tipe][key]['start'])
									else:
										row.append(el[p][tipe][key])
								elif tipe =='title':
									row.append(el[p][tipe][0]['plain_text'])
								elif tipe =='select':
									# print(el[p][tipe])
									row.append(el[p][tipe]['name'])
								elif tipe == 'rich_text':
									# print(el[p][tipe])
									row.append(el[p][tipe][0]['text']['content'])
								else:
									row.append(el[p][tipe])
							data.append(row)
					return data
	nsync = NotionSync()
	data = nsync.query_databases()
	projects = nsync.get_projects_titles(data[0])
	data = nsync.get_projects_data(data,projects)
	money_df = pd.DataFrame(data, columns = projects)
	return money_df

def main():
	money_df = getdata()
	money_df['Date'] = pd.to_datetime(money_df['Date'],format='%Y-%m-%dT%H:%M:%S.%f%z')
	st.title("My Spendings Report")
	menu = ["Monthly Spendings","All Time Spendings"]
	choice = st.selectbox('Menu',menu)
 	
	#All time report
	if choice == menu[1]:
		st.header("All Time Spendings")
		most_spend_cat = money_df[money_df['method']=='cash'].groupby('Category').sum().sort_values('price',ascending=False).iloc[0]

		st.subheader('Metrics')
		col1 = st.columns(2)
		col1[0].metric(label='Total Spendings',
							value=millify(money_df[money_df['method']=='cash']['price'].sum(), precision=2))
		col1[1].metric(label='Highest Spending Category',
							value=most_spend_cat.name.capitalize(),
							delta=millify(most_spend_cat.price, precision=2),
							delta_color='off')

		st.subheader('Spendings per Category')
		fig = px.bar(money_df[money_df['method']=='cash'].groupby('Category')['price'].sum().reset_index(),
								x='Category',y='price',text_auto=True,
								labels={'price':'Total Spendings'})
		fig.update_layout(xaxis={'categoryorder':'total ascending'})
		st.plotly_chart(fig, use_container_width=True)

		st.subheader('Spendings per Method')
		fig = px.bar(money_df.groupby('method')['price'].sum().reset_index(),
								x='method',y='price',text_auto=True,
								labels={'price':'Total Spendings'})
		fig.update_layout(xaxis={'categoryorder':'total ascending'})
		st.plotly_chart(fig, use_container_width=True)

		st.subheader('Spending Trends')
		fig = px.line(money_df[money_df['method']=='cash'].groupby(pd.Grouper(key='Date',freq='M'))['price'].sum().reset_index(), 
								x="Date", y="price", 
								labels={'price':'Total Spendings'})
		st.plotly_chart(fig, use_container_width=True)
	#Monthly Report
	elif choice == menu[0]:
		curr = pd.Timestamp.now()
		last_month = curr.replace(day=1) - pd.Timedelta(days=1)
		df = money_df.loc[money_df['Date'].dt.to_period(freq='M')== curr.to_period(freq='M') ].reset_index()
		last_month_df =  money_df.loc[money_df['Date'].dt.to_period(freq='M')== last_month.to_period(freq='M') ].reset_index()
		most_spend_cat = df[df['method']=='cash'].groupby('Category').sum().sort_values('price',ascending=False).iloc[0]

		st.header(f"Monthly Spendings {curr.strftime('%b %Y')}")

		st.subheader('Metrics')
		col1 = st.columns([1,1,2])
		col1[0].metric(label='This Month Spendings',
							value=millify(df[df['method']=='cash']['price'].sum(), precision=2),
							delta=f"{(df[df['method']=='cash']['price'].sum()-last_month_df[last_month_df['method']=='cash']['price'].sum())/last_month_df[last_month_df['method']=='cash']['price'].sum()*100:.2f}%",
							delta_color='inverse')
		col1[1].metric(label='Last Month Spendings',
							value=millify(last_month_df[last_month_df['method']=='cash']['price'].sum(), precision=2))
		col1[2].metric(label='Highest Spending Category',
							value=most_spend_cat.name.capitalize(),
							delta=millify(most_spend_cat.price, precision=2),
							delta_color='off')
		
		st.subheader('Spendings per Category')
		fig = px.bar(df[df['method']=='cash'].groupby('Category')['price'].sum().reset_index(),
								x='Category',y='price',text_auto=True,
								labels={'price':'Total Spendings'})
		fig.update_layout(xaxis={'categoryorder':'total ascending'})
		st.plotly_chart(fig, use_container_width=True)

		st.subheader('Spending Trends')
		fig = px.line(df[df['method']=='cash'].groupby(pd.Grouper(key='Date',freq=('D')))['price'].sum().reset_index().sort_values('Date'), 
								x="Date", y="price", 
								labels={'price':'Total Spendings'})
		st.plotly_chart(fig, use_container_width=True)
		#st.dataframe(df[['Name','price','method','Category','Date']])
	
if __name__ == '__main__':
	main()

Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pgrep streamlit

334
1731


In [ ]:
public_url = ngrok.connect(port='8501')

In [ ]:
public_url

'http://1ff6-34-80-6-99.ngrok.io'

In [ ]:
!kill 1731
ngrok.kill()